In [1]:
from __future__ import (absolute_import, division)
import numpy as np
from numpy.linalg import matrix_power
import matplotlib.pyplot as plt
%matplotlib inline

1.) Once the state transformation graph is drawn, it becomes apparent that $ \beta > 0$ makes the Markov chain irreducible.
We know that $\alpha$ and $\beta$ are probabilities and therefore they are less than or equal to 1. Assume $\beta>0 $ then we know certainly that states 0 and 2 communicate. If $(1-\alpha) >0 $ states 0 and 1 communicate via state 2 or if $ \alpha > 0 $ state 0 is accessible from state 1, directly. At least one of these two conditions for $\alpha$ will be true as its maximum value is 1 and the boundary conditions 0 and 1 both make all states communicate for all values of alpha at and within the boundary conditions if and only if $ \beta > 0$ . For the chain to be irreducible, all states must be communicating and therefore this is only possible when $\beta$ > 0

2.) Experimental solution via iterative algorithm

In [2]:
p = np.array([[1/2, 1/4, 1/4], [1/2, 0, 1/2], [1/4, 0, 3/4]])

In [3]:
def nth_power(m_mat, supscrt):
    tmp = m_mat
    old_tmp = tmp
    diff = []
    if(supscrt>=2):
        for i in range(2,supscrt):
            old_tmp = tmp
            tmp = np.dot(tmp, m_mat)
            diff.append(old_tmp - tmp)
        return tmp, diff
    else:
        raise ValueError('Power of matrix should be >= 2')

In [4]:
%timeit nth_power(p, int(1e5))

1 loop, best of 3: 175 ms per loop


In [5]:
m_res, m_diff = nth_power(p, int(1e5))

For sanity check, I ran the built-in numpy matrix power routine

In [6]:
%timeit matrix_power(p, int(1e5))

The slowest run took 9.92 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 28.2 µs per loop


In [7]:
np_res = matrix_power(p, int(1e5))

In [8]:
print(np_res, '\n \n', m_res)

[[ 0.36363636  0.09090909  0.54545455]
 [ 0.36363636  0.09090909  0.54545455]
 [ 0.36363636  0.09090909  0.54545455]] 
 
 [[ 0.36363636  0.09090909  0.54545455]
 [ 0.36363636  0.09090909  0.54545455]
 [ 0.36363636  0.09090909  0.54545455]]


Clearly, my algorithm calculates the power of the matrix correctly. I noticed I have rounding errors around 12th decimal but I don't think it matters given the nature of this question. My algorithm is correct, however significantly less efficient compared to the built-in numpy routine (3 orders of magnitude). The C bindings work much better for such numerical calculations, or even numexpr. 

The limiting probability $ \pi_j $ exists. Once we investigate how the transitional probabilities behave over various iterations by looking at the m_diff which simply saves the difference between two consecutive powers of transition matrix:

In [9]:
print('The difference between p and p^2\n', 
      m_diff[0])

The difference between p and p^2
 [[ 0.0625  0.125  -0.1875]
 [ 0.125  -0.125   0.    ]
 [-0.0625 -0.0625  0.125 ]]


In [10]:
print('The difference between p^99 and p^100\n', 
      m_diff[99])

The difference between p^99 and p^100
 [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


In [11]:
print('The difference between p^9998 and p^9999\n', 
      m_diff[9998])

The difference between p^9998 and p^9999
 [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


In other words, P^100k is close enough to the limiting possibility as $ lim_{n \rightarrow \infty}$ therefore $\pi_j$ exists.
Given the result from the nth_power function above, $ pi_j = lim_{n \rightarrow \infty}P_{i,j}^{(n)} $ can be estimated by $ \pi = [\pi_0, \pi_1, \pi_2] = [ 0.36363636, 0.09090909, 0.54545455] $ for state matrix {0 , 1, 2}

3.) Solving the system of linear equations:

$ \pi = \pi P$ where $P = [[1/2, 1/4, 1/4] [1/2, 0, 1/2], [1/4, 0, 3/4]]$

By writing out the matrix in linear equation form:

$ 0.5\pi_0 + 0.5\pi_1 + 0.25\pi_2 = \pi_0$

$ 0.25\pi_0 = \pi_1 $

$ 0.25\pi_0 + 0.5\pi_1 + 0.75\pi_2 = \pi_2$

$ \pi_0 + \pi_1 + \pi_2 = 1 $

4 equations and 3 unknowns makes this a simple solution giving us 

$ \pi_0 $ = 4/11 
$ \pi_1 $ = 1/11 
$ \pi_2 $ = 6/11 

which are identical to the results obtained in the earlier step.